In [7]:
import autogen

config_list = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        # "model": ["gpt4", "gpt-4-32k", "gpt-4-32k-0314", "gpt-4-32k-v0314"],
        "model": ["gpt-4"]
    },
)

llm_config={
    "request_timeout": 600,
    "seed": 42,
    "config_list": config_list,
    "temperature": 0,
}

In [8]:
import socket
# 获取主机名
hostname = socket.gethostname()
hostname


import random

# 定义一个字符串数组
str_array = ["string1", "string2", "string3", "string4", "string5"]

# 使用random.choice()从数组中随机选择一个元素
random_str = random.choice(str_array)

print(random_str)

from datetime import datetime

# 获取当前的日期和时间
current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")

print(current_time)


string4
2023-11-09 11:19:06.763208


In [9]:
llm_func_call_config={
    "request_timeout": 600,
    "seed": 42,
    "config_list": config_list,
    "temperature": 0,
    "functions": [
            {
                "name": "get_product_orders",
                "description": "run and call python script and return the execution result for get the most popular products in the given area",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "The location name",
                        }
                    },
                    "required": ["location"],
                },
            },
            {
                "name": "get_product",
                "description": "run and call python script and return the execution result for obtain the basic information and inventory quantity of the given product and location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "product": {
                            "type": "string",
                            "description": "The product name",
                        },
                        "location": {
                            "type": "string",
                            "description": "The location name",
                        }
                    },
                    "required": ["product", "location"],
                },
            },
        ],
}

In [10]:
def get_product_orders(location):
    import socket
    # 获取主机名
    hostName = socket.gethostname()
    import random

    # 定义一个字符串数组
    products = ["驴肉火烧", "肉夹馍", "烤白薯", "烤串", "蒜香卡布奇诺", "酱香拿铁"]

    # 使用random.choice()从数组中随机选择一个元素
    product = random.choice(products)

    r = { 
            "product": product,
            "orders": random.randint(1, 1000),
            "host": hostname,
            "location": location,
            "time": datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")
    }
    return r

def get_product(product, location):
    import socket
    # 获取主机名
    hostName = socket.gethostname()
    from datetime import datetime
    r = { 
            "product": product,
            "location": location,
            "stock": random.uniform(1.0, 10000.0), 
            # "stock": 88, 
            "host": hostname,
            "time": datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")
    }
    return r    



In [11]:
def is_termination_msg_func(x):
    r = x.get("content", "").rstrip().endswith("TERMINATE")
    print("===============", x)
    # r = True
    return r

code_execution_work_dir = "00_awesome_poc"

# create a UserProxyAgent instance named "user_proxy"
user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    # human_input_mode="TERMINATE",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=10,
    is_termination_msg=is_termination_msg_func, # lambda x: x.get("content", "").rstrip().endswith("TERMINATE"),
    llm_config=llm_config,
    # code_execution_config=False,
    code_execution_config={"work_dir": code_execution_work_dir},
    system_message="""Reply TERMINATE if the task has been solved at full satisfaction.
Otherwise, reply CONTINUE, or the reason why the task is not solved yet."""
)


planner = autogen.AssistantAgent(
    name="Planner",
    system_message='''
    Planner. 
    Suggest a plan. Revise the plan based on the execution result returned by the executor,
    The plan may involve an engineer who can write code and others don't write code.
    Explain the plan first. 
    To obtain the result of each step, you only need to call the corresponding function.
    Be clear which step is performed by an engineer,
    and which step is performed by an executor.
    ''',
    llm_config=llm_func_call_config,
    # llm_config=llm_config,
)

executor = autogen.UserProxyAgent(
    name="Executor",
    system_message="Executor. Execute the code written by the engineer and report the result.",
    human_input_mode="NEVER",
    code_execution_config={"last_n_messages": 3, "work_dir": code_execution_work_dir},
)

engineer = autogen.AssistantAgent(
    name="Engineer",
    # llm_config=llm_func_call_config,
    llm_config=llm_config,
    system_message='''Engineer. You follow the plan. You write python/shell code to solve tasks. 
    Wrap the code in a code block that specifies the script type. 
    The user can't modify your code. So do not suggest incomplete code which requires others to modify. 
    Don't use a code block if it's not intended to be executed by the executor.
    Don't include multiple code blocks in one response. Do not ask others to copy and paste the result. 
    Check the execution result returned by the executor.
    If the result indicates there is an error, fix the error and output the code again. 
    Suggest the full code instead of partial code or code changes. 
    If the error can't be fixed or if the task is not solved even after the code is executed successfully, 
    analyze the problem, revisit your assumption, 
    collect additional info you need, and think of a different approach to try.
    ''' ,

)

executor.register_function(
    function_map={
        "get_product_orders": get_product_orders,
        "get_product": get_product,
    }
)

In [18]:
groupchat = autogen.GroupChat(
    agents=[
    user_proxy,
    
    executor,
    planner,
    engineer,
    
    ], 
    messages=[], max_round=30)
    
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)

message="""
获取中国台湾的最受欢迎的商品及其在该地区的库存
"""

message="""
请告诉我在中国台湾最受欢迎的产品在该地区还有多少货?
"""

# message="""
# 获取最受欢迎的商品
# """

# message=f"""
#     获取最受欢迎的商品的库存
# """

# message=f"""
#     获取订单销量最少的商品的库存及订单笔数
# """

# message=f"""
#     获取订单销量最少的商品
# """

# message=f"""
#     获取订单最多的商品
# """

user_proxy.initiate_chat(
    manager,
    message=message
)

user_proxy (to chat_manager):


请告诉我在中国台湾最受欢迎的产品在该地区还有多少货?


--------------------------------------------------------------------------------
Planner (to chat_manager):

首先，我们需要执行两个步骤来获取这个信息。第一步是找出在台湾最受欢迎的产品，第二步是查找该产品在台湾的库存数量。

第一步将由执行器执行，它将调用一个Python脚本来获取在台湾最受欢迎的产品。这个脚本将从我们的数据库中获取所有在台湾销售的产品的订单数量，然后返回订单数量最多的产品。

第二步也将由执行器执行，它将调用另一个Python脚本来获取在台湾最受欢迎的产品的库存数量。这个脚本将从我们的库存数据库中获取该产品在台湾的库存数量。

这两个步骤都不需要工程师的参与，因为它们都是通过执行器调用Python脚本来完成的。

现在，让我们开始执行这个计划。
***** Suggested function Call: get_product_orders *****
Arguments: 
{
  "location": "台湾"
}
*******************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION get_product_orders...
Executor (to chat_manager):

***** Response from calling function "get_product_orders" *****
{'product': '驴肉火烧', 'orders': 97, 'host': 'AwesomeYuer-P1', 'location': '台湾', 'time': '2023-11-09 12:02:52.697173'}
************************************************************